In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv(r'../input/train.csv',dtype = np.float32)

targets = train.label.values
features = train.loc[:,train.columns != "label"].values/255

features_train, features_test, targets_train, targets_test = train_test_split(features, targets, test_size = 0.2, random_state=42)

featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor) # data type is long

featuresTest = torch.from_numpy(features_test)
targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor) # data type is long

print(featuresTrain[1].shape, featuresTest.shape)
print(targetsTrain.shape, targetsTest.shape)

In [ ]:
#Create CNN model
from torch.autograd import Variable
import torch.nn.functional as F


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.cnn3 = nn.Conv2d(32, 64, 3)
        self.cnn4 = nn.Conv2d(64, 64, 3)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout2d(p=0.2)
        self.fc1 = nn.Linear(4*4*64, 70)
        self.fc2 = nn.Linear(70,10)
        
    def forward(self, x):
        out_con1 = self.cnn1(x)
        out_relu1 = self.relu(out_con1)
        out_con2 = self.cnn2(out_relu1)
        out_relu2 = self.relu(out_con2)
        
        out_pool1 = self.maxpool(out_relu2)
        
        out_con3 = self.cnn3(out_pool1)
        out_relu3 = self.relu(out_con3)
        out_con4 = self.cnn4(out_relu3)
        out_relu4 = self.relu(out_con4)
        
        out_maxpool = self.maxpool(out_relu4)
        out_dropout2 = self.dropout(out_maxpool)
        
        out = out_dropout2.view(out_dropout2.size(0), -1)
        out = self.fc1(out)
        
        return self.fc2(out)

batch_size = 100
#interations = 2500
epochs = 5000
#print(epochs)
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

model = CNNModel()
model.cuda()

error = nn.CrossEntropyLoss()

learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, betas=(0.9, 0.999))


In [ ]:
def adjust_learning_rate(optimizer):
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr'] * 0.9


count = 0
#loss_list = []
#iteration_list = []
#accuracy_list = []
for epoch in range(1, epochs+1):
    for i, (images, labels) in enumerate(train_loader):
        train = Variable(images.view(batch_size, 1, 28, 28)).cuda()
        labels = Variable(labels).cuda()
        
        optimizer.zero_grad()
        
        outputs = model(train)
        
        loss = error(outputs, labels).cuda() 
        loss.backward()
        optimizer.step()
        count += 1
    if epoch % 10 == 0:
        correct = 0
        total = 0
        for images, labels in test_loader:
            test = Variable(images.view(batch_size,1,28,28)).cuda()
            labels = Variable(labels).cuda()
            outputs = model(test)
            predicted = torch.max(outputs.data, 1)[1].cuda().data.squeeze()
            total += len(labels)
            correct += (predicted == labels).sum()  
        accuracy = 100 * float(correct) / float(total)

        #loss_list.append(loss.data)
        #iteration_list.append(count)
        #accuracy_list.append(accuracy)
        print('Epoch: {}  Loss: {}  Accuracy: {} %'.format(epoch, loss.data[0], round(accuracy, 7)))       

In [ ]:
test_csv = pd.read_csv(r'../input/test.csv',dtype = np.float32)
testfeatures = test_csv.loc[:,test_csv.columns != "label"].values/255

featurestest = torch.from_numpy(testfeatures)

ans_arr = []
for i, images in enumerate(featurestest):
    images = Variable(images.view(-1, 1, 28, 28)).cuda()
    outputs = model(images)
    #print(outputs.size())
    _, predicted = torch.max(outputs.data, 1)
    ans_arr.append(predicted)


In [ ]:
ans_arr = [int(x) for x in ans_arr]
df_preds = pd.DataFrame()
df_preds['ImageId'] = pd.Series([i+1 for i in range(28000)])
df_preds['Label'] = pd.Series(ans_arr)
df_preds.to_csv("base.csv", index=False)